# Tensorflow on RNA sequences

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# create dataset object that holds features and labels and cycles through the data in batches
class Dataset(object):

    def __init__(self, features, labels):
        assert (len(features) == len(labels))
        self.features = np.array(features)
        self.labels = np.array(labels)
        self.index = 0
        self.size = len(labels)
    
    def next_batch(self, batch_size):
        old_index = self.index
        new_index = self.index + batch_size
        self.index = new_index % self.size
        if new_index <= self.size:
            return (self.features[old_index: new_index], self.labels[old_index: new_index])
        else:
            subfeatures = np.concatenate([self.features[old_index:], self.features[:self.index]])
            sublabels = np.concatenate([self.labels[old_index:], self.labels[:self.index]])
            return (subfeatures, sublabels)
    
    def reset_index(self):
        self.index = 0
    

### Import MNIST data for testing

In [7]:
# import sample data for digit recognition
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [8]:
mnist_train = Dataset(mnist.train.images, mnist.train.labels)
mnist_test = Dataset(mnist.test.images, mnist.test.labels)

## Convolutional Neural Network

In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

def train_model(sess, train_step, eval_var, num_epoch, batch_size, report_int, keep_prob_train, train, test):

    # initialize variables
    sess.run(tf.global_variables_initializer())

    # train epochs
    for i in range(num_epoch):
        batch = train.next_batch(50)
        if i%report_int == 0:
            train_accuracy = eval_var.eval(feed_dict={x:batch[0],
                                                      y_: batch[1],
                                                      keep_prob: 1.0})

            print("step %d, training accuracy %g"%(i, train_accuracy),
                  "test accuracy %g"%eval_var.eval(feed_dict={x: test.features,
                                                              y_: test.labels,
                                                              keep_prob: 1.0}))
        train_step.run(feed_dict={x: batch[0],
                                  y_: batch[1],
                                  keep_prob: keep_prob_train})

    print("test accuracy %g"%eval_var.eval(feed_dict={x: test.features,
                                                      y_: test.labels,
                                                      keep_prob: 1.0}))
    

### First implement MNIST with a convolutional NN using tensorflow tutorial

In [11]:
init_size = 28
final_size = 7
num_output1 = 32
num_output2 = 64
fully_connected_nodes = 1024
out_nodes = 10

# create placeholders for data
x = tf.placeholder(tf.float32, shape=[None, init_size*init_size])
x_image = tf.reshape(x, [-1,init_size,init_size,1])
y_ = tf.placeholder(tf.float32, shape=[None, out_nodes])

# add convolution that traverses every 4x4 square, without overlaps
W_conv1 = weight_variable([4, 4, 1, num_output1])
b_conv1 = bias_variable([num_output1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# add a second convolution that looks at every 2x2 square, with overlaps
W_conv2 = weight_variable([2, 2, num_output1, num_output2])
b_conv2 = bias_variable([num_output2])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

# add a fully connected layer
W_fc1 = weight_variable([final_size * final_size * num_output2, fully_connected_nodes])
b_fc1 = bias_variable([fully_connected_nodes])

h_pool2_flat = tf.reshape(h_pool2, [-1, final_size * final_size * num_output2])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# add dropout to reduce overfitting
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([fully_connected_nodes, out_nodes])
b_fc2 = bias_variable([out_nodes])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2


In [17]:
# use a cross entropy loss function and optimize with Adam
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

train_model(sess, train_step, accuracy, 1000, 50, 100, mnist_train, mnist_test)


step 0, training accuracy 0.08 test accuracy 0.0731
step 100, training accuracy 0.66 test accuracy 0.7809
step 200, training accuracy 0.9 test accuracy 0.8655
step 300, training accuracy 0.92 test accuracy 0.9016
step 400, training accuracy 0.9 test accuracy 0.915
step 500, training accuracy 0.94 test accuracy 0.9169
step 600, training accuracy 0.94 test accuracy 0.9289
step 700, training accuracy 1 test accuracy 0.9345
step 800, training accuracy 0.92 test accuracy 0.9444
step 900, training accuracy 0.98 test accuracy 0.9393
test accuracy 0.9464


## Classify RNA sequences based on complementarity

In [3]:
def generate_random_sequence(length):
    """Generate a random RNA sequence of a given length"""
    
    nts = ['A','U','C','G']
    sequence = np.random.choice(nts, size=length, replace=True)

    return ''.join(sequence)

def get_complementary(seq):
    """Get the complementary sequence of a given RNA sequence"""
    
    intab = "AUCG"
    outtab = "UAGC"
    trantab = str.maketrans(intab, outtab)

    return seq.translate(trantab)

def generate_match_pair(length, random_seed=None):
    """Generate two sequences that are base-paired"""
    
    if random_seed is not None:
        np.random.seed(random_seed)

    seq1 = generate_random_sequence(length)
    seq2 = get_complementary(seq1)
    
    return seq1, seq2

def generate_seed_match_pair(length1, length2, random_seed=None):
    """Generate two sequences that are base-paired at positions 1-7"""
    
    if random_seed is not None:
        np.random.seed(random_seed)

    seq1 = generate_random_sequence(length1)
    up_fragment = generate_random_sequence(1)
    down_fragment = generate_random_sequence(length2-7)
    mid_fragment = get_complementary(seq1[1:7])
    
    seq2 = up_fragment + mid_fragment + down_fragment
    
    return seq1, seq2

def generate_random_pair(length1, length2, random_seed=None):
    """Generate two random sequences that are not perfectly complementary"""
    
    if random_seed is not None:
        np.random.seed(random_seed)
    
    seq1 = generate_random_sequence(length1)
    match_seq1 = get_complementary(seq1)

    while True:
        seq2 = generate_random_sequence(length2)

        if match_seq1 != seq2:
            return seq1, seq2

def one_hot_encode(seq, nt_order):
    """Convert RNA sequence to one-hot encoding"""
    
    one_hot = [list(np.array(nt_order == nt, dtype=int)) for nt in seq]
    one_hot = [item for sublist in one_hot for item in sublist]
    
    return np.array(one_hot)

def make_square(seq1, seq2):
    """Given two sequences, calculate outer product of one-hot encodings"""

    return np.outer(one_hot_encode(seq1, np.array(['A','U','C','G'])),
                    one_hot_encode(seq2, np.array(['U','A','G','C'])))

In [5]:
print(generate_seed_match_pair(20,20))
print(generate_random_pair(20,20))
print(generate_match_pair(20,20))
print(make_square('AAA','AAA'))
print(make_square('UAG','AUC'))

('CUAGCUCCUGAGGACUACUC', 'AAUCGAGCAGUCUCGAUAGU')
('CAUCCUAUAGCCUACUCGGC', 'GCAACGUCGAAUCUUGGAGG')
('GCGGACUAGCGCACAUCCGG', 'CGCCUGAUCGCGUGUAGGCC')
[[0 1 0 0 0 1 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 1 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 1 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 1 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 1 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 1 0 0 0 0 0 0 1]]


In [12]:
def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

# make 2D neural network object
class NeuralNet2D(object):
    
    def __init__(self, sess, dim1, dim2, label_size):
        """Initiate object with placeholders for data and layers"""
        self.sess = sess
        self.x = tf.placeholder(tf.float32, shape=[None, dim1*dim2])
        self.y_ = tf.placeholder(tf.float32, shape=[None, label_size])
        
        print(self.x.get_shape())
        
        self.x_image = tf.summary.image('images', tf.reshape(self.x, [-1, dim1, dim2, 1]))
        
        self.layers = [tf.reshape(self.x, [-1,dim1,dim2,1])]
        self.layer_index = 0
    
    def add_layer(self, input_tensor, weight_dim, output_dim, layer_name, preact, act):
        """Reusable code for making a simple neural net layer.

        It does a matrix multiply, bias add, and then uses an activation function to nonlinearize.
        It also sets up name scoping so that the resultant graph is easy to read,
        and adds a number of summary ops.
        """
        # add a name scope ensures logical grouping of the layers in the graph.
        with tf.name_scope(layer_name):
            # create variables for weights and biases
            with tf.name_scope('weights'):
                weights = weight_variable(weight_dim)
                variable_summaries(weights)
            with tf.name_scope('biases'):
                biases = bias_variable([output_dim])
                variable_summaries(biases)
            with tf.name_scope('Wx_plus_b'):
                preactivate = preact(input_tensor, weights, biases)
                tf.summary.image('pre_activations', preactivate)
            
            out_layer = act(preactivate, name='activation')
            tf.summary.histogram('activations', out_layer)
            
            return out_layer
    
    def add_convolution(self, layer_name, dim1, dim2, stride1, stride2,
                        output_channels, padding='SAME', act=tf.nn.relu):
        """Create a convolution layer. 
        Dim1 and dim2 specify the size of the box for the convolution
        """
        # get the current layer and determine how many output channels it has
        current_layer = self.layers[self.layer_index]

        # this becomes the new input channel size
        input_channels = current_layer.get_shape().as_list()[-1]
        
        # specify the function that creates a new tensor
        preact = lambda tensor, weights, biases: (tf.nn.conv2d(tensor,
                                                               weights,
                                                               strides=[1, stride1, stride2, 1],
                                                               padding=padding) + biases)

        # create the new tensor
        new_layer = self.add_layer(current_layer,
                                   [dim1, dim2, input_channels, output_channels],
                                   output_channels, layer_name, preact, act)

        self.layers.append(new_layer)
        self.layer_index += 1

    def add_max_pool(self, dim1, dim2, stride1, stride2, padding='SAME'):
        """Adds a max pooling layer."""
        self.layers.append(tf.nn.max_pool(self.layers[self.layer_index],
                                            ksize=[1, dim1, dim2, 1],
                                            strides=[1, stride1, stride2, 1], padding=padding))
        
        self.layer_index += 1

    def add_fully_connected(self, layer_name, output_channels, act=tf.nn.relu):
        """Adds a fully connected layer."""

        current_layer = self.layers[self.layer_index]
        dim = current_layer.get_shape().as_list()
        dim = dim[1] * dim[2] * dim[3]
        
        preact = lambda tensor, weights, biases: tf.matmul(tensor, weights) + biases
    
        new_layer = self.add_layer(tf.reshape(current_layer, [-1, dim]),
                                   [dim, output_channels],
                                   output_channels, layer_name, preact, act)
            
        self.layers.append(new_layer)
        self.layer_index += 1
    
    def add_dropout(self, layer_name, num_nodes):
        """Adds a layer for dropout nodes in order to reduce overfitting"""
        current_layer = self.layers[self.layer_index]
        dim = current_layer.get_shape().as_list()
        self.keep_prob = tf.placeholder(tf.float32)
        
        with tf.name_scope(layer_name):
            with tf.name_scope('weights'):
                weights = weight_variable([dim[-1], num_nodes])
                variable_summaries(weights)
            with tf.name_scope('biases'):
                biases = bias_variable([num_nodes])
                variable_summaries(biases)
            with tf.name_scope('Wx_plus_b'):
                out_layer = tf.matmul(tf.nn.dropout(current_layer, self.keep_prob), weights) + biases

            tf.summary.histogram('activations', out_layer)

        self.layers.append(out_layer)
        self.layer_index += 1
    
    def make_train_step(self, problem_type, logdir):
        current_layer = self.layers[self.layer_index]
        if problem_type == 'classification':
            cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(current_layer,
                                                                                   self.y_))
            with tf.name_scope('train'):
                self.train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
            
            correct_prediction = tf.equal(tf.argmax(current_layer,1), tf.argmax(self.y_,1))
            
            with tf.name_scope('accuracy'):
                self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            
        elif problem_type == 'regression':
            SS_err = tf.reduce_sum(tf.square(tf.sub(current_layer, self.y_)))
            SS_tot = tf.reduce_sum(tf.square(tf.sub(self.y_, tf.reduce_mean(self.y_))))
            R_2 = tf.sub(tf.cast(1.0, tf.float32), tf.div(SS_err, SS_tot))

            with tf.name_scope('train'):
                self.train_step = tf.train.AdamOptimizer(1e-4).minimize(SS_err)
            with tf.name_scope('accuracy'):
                self.accuracy = R_2
        
        else:
            print('problem_type must be \'classification\' or \'regression\'')
            
        tf.summary.scalar('accuracy', self.accuracy)
        
        self.merged = tf.summary.merge_all()
        self.train_writer = tf.summary.FileWriter(logdir + '/train', self.sess.graph)
        self.test_writer = tf.summary.FileWriter(logdir + '/test')
    
    def train_model(self, train, test, num_epoch=20000, batch_size=50, 
                    report_int=1000, keep_prob_train=0.5):

        # initialize variables
        self.sess.run(tf.global_variables_initializer())
        

        # train epochs
        for i in range(num_epoch):
            batch = train.next_batch(batch_size)
            if i%report_int == 0:
                
                acc, summary = self.sess.run([self.accuracy, self.merged], feed_dict={self.x: test.features,
                                                                                 self.y_: test.labels,
                                                                                 self.keep_prob: 1.0})
                self.test_writer.add_summary(summary, i)
#                 print('Accuracy at step %s: %s' % (i, acc))
            
            else:
                _, summary = self.sess.run([self.train_step, self.merged], feed_dict={self.x: batch[0],
                                                                                 self.y_: batch[1],
                                                                                 self.keep_prob: keep_prob_train})
                self.train_writer.add_summary(summary, i)

        print("test accuracy %g"%self.accuracy.eval(feed_dict={self.x: test.features,
                                                               self.y_: test.labels,
                                                               self.keep_prob: 1.0}))


### Learn to distinguish complementary sequences from random sequences

In [6]:
# generate complementary and random 4mers
features = np.zeros((1000, 256))
labels = np.zeros((1000, 2))

for i in range(1000):
    if np.random.random() < 0.5:
        seq1, seq2 = generate_match_pair(4)
        labels[i,:] = [1, 0]
    else:
        seq1, seq2 = generate_random_pair(4,4)
        labels[i,:] = [0, 1]

    features[i,:] = make_square(seq1, seq2).flatten()

match_train = Dataset(features[:900, :], labels[:900, :])
match_test = Dataset(features[900:, :], labels[900:, :])

In [13]:
with tf.Graph().as_default(), tf.Session() as sess:

    dim1, dim2 = 16, 16
    num_output1 = 4
    fully_connected_nodes = 1024
    out_nodes = 2

    NN = NeuralNet2D(sess, dim1, dim2, out_nodes)
    NN.add_convolution('layer1', 4, 4, 4, 4, num_output1)
    NN.add_fully_connected('layer2', fully_connected_nodes)
    NN.add_dropout('layer3', out_nodes)
    NN.make_train_step('classification', '../logdirs/match')
    NN.train_model(match_train, match_test, num_epoch=1000,
                   batch_size=50, report_int=10, keep_prob_train=0.5)


(?, 256)
test accuracy 0.97


### Learn to distinguish seed-matched sequences from random sequences

In [96]:
# generate mirna,utr pairs of length 20 with a seed match
features = np.zeros((1000, 6400))
labels = np.zeros((1000,2))

for i in range(1000):
    if np.random.random() < 0.5:
        seq1, seq2 = generate_seed_match_pair(20,20)
        labels[i,:] = [1, 0]
    else:
        seq1, seq2 = generate_random_pair(20,20)
        labels[i,:] = [0, 1]
    
    features[i,:] = make_square(seq1, seq2).flatten()

seed_match_train = Dataset(features[:900, :], labels[:900, :])
seed_match_test = Dataset(features[900:, :], labels[900:, :])

In [19]:
dim1, dim2 = 80, 80
num_output1 = 4
fully_connected_nodes = 1024
out_nodes = 2

NN = NeuralNet2D(sess, dim1, dim2, out_nodes)
NN.add_convolution(4, 4, 4, 4, num_output1)
NN.add_fully_connected(fully_connected_nodes)
NN.add_dropout(out_nodes)
NN.make_train_step('classification')
NN.train_model(seed_match_train, seed_match_test, num_epoch=5000,
               batch_size=50, report_int=1000, keep_prob_train=0.5)

TypeError: add_convolution() missing 1 required positional argument: 'output_channels'

### Learn to regress on seed pairing stability

In [19]:
def calculate_sps(seq):
    """
    Parameters:
    ==========
    seq: string, consist of A, U, C, G
    
    Returns:
    =======
    float: seed pairing stability
    """
    thermo_dict = {'AA':-0.93,'AU':-1.10,'AC':-2.24,'AG':-2.08,
               'UA':-1.33,'UU':-0.93,'UC':-2.35,'UG':-2.11,
               'CA':-2.11,'CU':-2.08,'CC':-3.26,'CG':-2.36,
               'GA':-2.35,'GU':-2.24,'GC':-3.42,'GG':-3.26}
    init = 4.09
    terminal_au = 0.45
    
    # initialize score
    score = init
    
    # add score for each dinucleotide
    for i in range(len(seq)-1):
        score += thermo_dict[seq[i:i+2]]
    
    # add score for each terminal AU
    score += terminal_au*((seq[0]+seq[-1]).count('A') + (seq[0]+seq[-1]).count('U'))
    
    return score


In [106]:
# generate random sequences
features = np.zeros((1000, 6400))
labels = np.zeros((1000,1))
seq1s, seq2s = [], []

for i in range(1000):
    if np.random.random() < 0.5:
        seq1, seq2 = generate_seed_match_pair(20,20)
        seq1s.append(seq1)
        seq2s.append(seq2)
        labels[i,:] = calculate_sps(seq1[1:7]) + (np.random.random()-0.5)
    else:
        seq1, seq2 = generate_random_pair(20,20)
        seq1s.append(seq1)
        seq2s.append(seq2)
        labels[i,:] = np.random.random()-0.5
    
    features[i,:] = make_square(seq1, seq2).flatten()

seed_match_sps_train = Dataset(features[:900, :], labels[:900, :])
seed_match_sps_test = Dataset(features[900:, :], labels[900:, :])

In [110]:
dim1, dim2 = 80, 80
num_output1 = 16
num_output2 = 16
fully_connected_nodes = 32
out_nodes = 1

NN = NeuralNet2D(sess, dim1, dim2, out_nodes)
NN.add_convolution(4, 4, 4, 4, num_output1)
NN.add_convolution(2, 2, 1, 1, num_output2)
NN.add_fully_connected(fully_connected_nodes)
NN.add_dropout(out_nodes)
NN.make_train_step('regression')
NN.train_model(seed_match_sps_train, seed_match_sps_test, num_epoch=10000,
               batch_size=100, report_int=1000, keep_prob_train=0.5)

step 0, training accuracy -1.04116 test accuracy -0.901628
step 1000, training accuracy 0.540802 test accuracy 0.367427
step 2000, training accuracy 0.942358 test accuracy 0.861274
step 3000, training accuracy 0.981681 test accuracy 0.913068
step 4000, training accuracy 0.988818 test accuracy 0.926939
step 5000, training accuracy 0.987021 test accuracy 0.925711
step 6000, training accuracy 0.985119 test accuracy 0.928617
step 7000, training accuracy 0.99206 test accuracy 0.936477
step 8000, training accuracy 0.996499 test accuracy 0.941244
step 9000, training accuracy 0.994621 test accuracy 0.942679
test accuracy 0.944907


# Regression on real data

In [14]:
import pandas as pd

LOGFC_FILE1 = '../data/Supplementary1.csv'
LOGFC_FILE2 = '../data/Supplementary2.csv'
GENE_FILE = '../data/Gene_info.txt'
SEED_FILE = '../data/seed_dict.csv'
SEQ_FILE = '../data/seq_dict.csv'

def rev_comp(seq):
    """Get the reverse complement of a given RNA sequence"""
    
    intab = "AUCG"
    outtab = "UAGC"
    trantab = str.maketrans(intab, outtab)

    return seq.translate(trantab)[::-1]
    
rev_comp('AUUACCGGC')

'GCCGGUAAU'

In [15]:
GENE_INFO = pd.read_csv(GENE_FILE,sep='\t').drop(['Gene description','Species ID'],1)
GENE_INFO = GENE_INFO.groupby('Gene symbol').agg(lambda x:tuple(x))
GENE_INFO.loc[:,'Isoform ratio'] = [float(max(x))/np.nansum(x) for x in GENE_INFO['3P-seq tags + 5']]
GENE_INFO.loc[:,'Transcript ID'] = [x[y.index(1)] for (x,y) in zip(GENE_INFO['Transcript ID'],GENE_INFO['Representative transcript?'])]
GENE_INFO = GENE_INFO[['Transcript ID','Isoform ratio']]
GENE_INFO_TRANSCRIPT = GENE_INFO.reset_index().set_index('Transcript ID')

SEED_INFO = pd.read_csv(SEED_FILE,sep='\t')
SEED_DICT = {}
for row in SEED_INFO.iterrows():
    SEED_DICT[row[1]['col']] = row[1]['seed']

SEEDS = sorted(list(SEED_DICT.values()))

SEQ_INFO = pd.read_csv(SEQ_FILE, sep='\t')
SEQ_INFO['seed'] = [SEED_DICT[x] for x in SEQ_INFO['col']]
SEQ_INFO = SEQ_INFO.set_index('seed')
SEQ_INFO['mirna_seq'] = [(x + 'AAAAAAA')[:24] for x in SEQ_INFO['mirna_seq']]
SEQ_INFO.head()

,col,mirna_seq
seed,,
UCGUAGG,1595297366,UUCGUAGGUCAAAAUACACAAAAA
UCAUCUC,1595297383,UUCAUCUCCAAUUCGUAGGAAAAA
UGCUCUU,1595297389,AUGCUCUUUCCUCCUGUGCAAAAA
UUUGGAA,1595297394,UUUUGGAACAGUCUUUCCGAAAAA
UUGGAAC,1595297399,UUUGGAACAGUCUUUCCGAAAAAA


In [64]:
refseq_gene = pd.read_csv('../../05_TargetPrediction/data/refseq_gene.txt',sep='\t').set_index('query')
print(len(GENE_INFO))
refseq_gene.head()

19432


,symbol
query,
NM_001003806,TARP
NM_001003805,ATP5S
NM_001003802,SMARCD3
NM_001003803,ATP5S
NM_001003800,BICD2


In [65]:
GENE_INFO.head()

,Transcript ID,Isoform ratio
Gene symbol,,
0,ENST00000401030.3,0.333333
A1BG,ENST00000263100.3,1.000000
A1CF,ENST00000374001.2,1.000000
A2M,ENST00000318602.7,1.000000
A2ML1,ENST00000299698.7,1.000000


In [83]:
expression = pd.read_csv('../../05_TargetPrediction/data/GSE52530_HeLa.expData.qn.txt.gz',sep='\t')[['#geneID','geneLen','Puc19_1,Puc19_2']]
expression['Average expression'] = [np.mean([float(x) for x in exp.split(',')]) for exp in list(expression['Puc19_1,Puc19_2'])]
# expression['RPM'] = (expression['Average expression'] * expression['geneLen'])/ 1000.0
expression = expression.drop(['geneLen','Puc19_1,Puc19_2'],1)

expression['Gene symbol'] = list(refseq_gene.loc[expression['#geneID']]['symbol'])
print(len(expression))
expression = expression.dropna().groupby('Gene symbol').agg(np.nanmean)
print(len(expression))
expression = pd.concat([expression, GENE_INFO], axis=1, join='inner')
print(len(expression))
expression = expression.reset_index().groupby('Transcript ID').first().drop('Gene symbol',1)
print(len(expression))
expression.head()

35956
22351
18200
18200


,Average expression,Isoform ratio
Transcript ID,,
ENST00000000233.5,45.384213,1.0
ENST00000000412.3,27.196628,1.0
ENST00000001008.4,35.387227,1.0
ENST00000001146.2,1.151177,1.0
ENST00000002125.4,4.902705,1.0


In [85]:
np.percentile(expression['Average expression'], 60)

4.0205855781509987

In [98]:
UTRS = pd.read_csv('../../05_TargetPrediction/targetscan_files/UTR_Sequences_Ensembl_Human.txt',
                   sep='\t', usecols=['Ensembl ID','UTR sequence']).set_index('Ensembl ID')
UTRS['UTR sequence'] = [x.replace('-','').upper().replace('T','U') for x in UTRS['UTR sequence']]
UTRS['UTR length'] = [len(x) for x in UTRS['UTR sequence']]
# UTRS = pd.concat([UTRS, expression], axis=1, join='inner')
UTRS.head()

,UTR sequence,UTR length
Ensembl ID,,
CDR1as,GUUUCCGAUGGCACCUGUGUCAAGGUCUUCCAACAACUCCGGGUCU...,1485
ENST00000000233.5,CCAGCCAGGGGCAGGCCCCUGAUGCCCGGAAGCUCCUGCGUGCAUC...,422
ENST00000000412.3,AUUGCACUUUAUAUGUCCAGCCUCUUCCUCAGUCCCCCAAACCAAA...,1457
ENST00000001008.4,CCCCUCUCCACCAGCCCUACUCCUGCGGCUGCCUGCCCCCCAGUCU...,2163
ENST00000001146.2,CCCAAGACCCACCCGCCUCAGCCCAGCCCAGGCAGCGGGGUGGUGG...,3001


In [112]:
logFCs1 = pd.read_csv(LOGFC_FILE1)
# logFCs1 = logFCs1[logFCs1['Used in training'] == 'yes']
logFCs1['Transcript ID'] = list(GENE_INFO.loc[logFCs1['Gene symbol']]['Transcript ID'])
logFCs1 = logFCs1.drop(['Used in training','RefSeq ID','Gene symbol'],1).dropna(subset=['Transcript ID'])
logFCs1 = logFCs1.drop_duplicates(subset=['Transcript ID']).set_index('Transcript ID')
logFCs1.columns = [SEED_DICT[x] if x in SEED_DICT else x for x in logFCs1.columns]

logFCs2 = pd.read_csv(LOGFC_FILE2)
logFCs2['Transcript ID'] = list(GENE_INFO.loc[logFCs2['Gene symbol']]['Transcript ID'])
logFCs2 = logFCs2.drop(['RefSeq ID','Gene symbol'],1).dropna(subset=['Transcript ID'])
logFCs2 = logFCs2.drop_duplicates(subset=['Transcript ID']).set_index('Transcript ID')
logFCs2.columns = [SEED_DICT[x] if x in SEED_DICT else x for x in logFCs2.columns]

logFCs = pd.concat([logFCs1,logFCs2],axis=1,join='outer')
# logFCs = pd.concat([logFCs, GENE_INFO_TRANSCRIPT], axis=1, join='inner')
logFCs = pd.concat([logFCs, expression, UTRS[['UTR length']]], axis=1, join='inner')

logFCs = logFCs[logFCs['Isoform ratio'] > 0.9]
logFCs = logFCs[logFCs['Average expression'] > np.percentile(expression['Average expression'], 50)]
print(len(logFCs))
logFCs.head()

5672


,UCGUAGG,UCAUCUC,UGCUCUU,UUUGGAA,UUGGAAC,CAAACAC,AAUACAC,UUUCCUC,AGCUUCC,AGUCAGA,...,AGCAGCA,AAAGUGC,AACACUG,AAUACUG,UGACCUA,GAGGUAG,GCAGCAU,Average expression,Isoform ratio,UTR length
ENST00000000233.5,0.069,0.018,0.048,-0.033,-0.076,-0.007,0.059,0.055,-0.019,0.065,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.384213,1.000000,422
ENST00000000412.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.098,-0.514,-0.056,0.031,-0.003,0.039,0.078,27.196628,1.000000,1457
ENST00000001008.4,-0.016,-0.557,0.110,0.241,-0.231,0.054,-0.031,0.160,-0.021,0.099,...,0.016,0.133,0.007,-0.114,-0.166,0.002,0.038,35.387227,1.000000,2163
ENST00000002165.6,-0.058,0.092,0.036,0.038,-0.191,0.067,0.057,0.046,-0.055,-0.075,...,0.063,0.024,-0.018,0.178,0.015,0.031,-0.176,24.530939,0.999025,1774
ENST00000002829.3,0.461,0.208,0.725,0.168,0.003,-0.209,0.182,-0.088,0.063,0.112,...,-0.205,0.032,0.146,-0.240,0.064,0.014,NaN,23.912061,1.000000,960


### Create datasets with site type and SPS

In [40]:
zipped = list(zip(SEEDS, [rev_comp(seed) for seed in SEEDS], [SEQ_INFO.loc[seed]['mirna_seq'] for seed in SEEDS]))
features, labels, sequences = [], [], []

for row in logFCs.iterrows():
    utr = UTRS.loc[row[0]]['UTR sequence']
    row = row[1]
    for seed, rev, mirna in zipped:
        val = row[seed]
        if np.isnan(val):
            continue
        elif utr.count(rev[1:]) == 1:
            loc = utr.find(rev[1:])
            if (loc-15) >= 0:
                if (loc + 9) < len(utr):
                    seq1, seq2 = mirna[:20], utr[loc-15:loc+9]
                    sequences.append((seq1, seq2[::-1]))
                    features.append(make_square(seq1, seq2[::-1]).flatten())
                    if (rev + 'A') in seq2:
                        labels.append(-3 + (calculate_sps(rev)/10.0))
                    elif (rev) in seq2:
                        labels.append(-2 + (calculate_sps(rev)/10.0))
                    elif (rev[1:] + 'A') in seq2:
                        labels.append(-2 + (calculate_sps(rev[1:])/10.0))
                    elif (rev[1:]) in seq2:
                        labels.append(-1)
                    else:
                        print('blah', seq2, rev)
                        break


features = np.array(features)
labels = np.array(labels).reshape(len(labels), 1)
test_size = int(len(features)/10)
constructed_logfc_train = Dataset(features[test_size:], labels[test_size:])
constructed_logfc_test = Dataset(features[:test_size], labels[:test_size])
labels.shape

(42837, 1)

In [38]:
features.shape

(42837, 7680)

In [41]:
with open('../data/constructed_logfc_100.txt', 'w') as f:
    i = 0
    for seqs, label in zip(sequences, labels):
        f.write('{},{},{}\n'.format(seqs[0], seqs[1], label[0]))
        i += 1
        if i == 100:
            break

In [123]:
dim1, dim2 = 80, 96
num_output1 = 16
num_output2 = 16
fully_connected_nodes = 32
out_nodes = 1

NN = NeuralNet2D(sess, dim1, dim2, out_nodes)
NN.add_convolution(4, 4, 4, 4, num_output1)
NN.add_convolution(2, 2, 1, 1, num_output2)
NN.add_fully_connected(fully_connected_nodes)
NN.add_dropout(out_nodes)
NN.make_train_step('regression')
NN.train_model(constructed_logfc_train, constructed_logfc_test, num_epoch=10000,
               batch_size=100, report_int=1000, keep_prob_train=0.5)

step 0, training accuracy -4.88184 test accuracy -4.65476
step 1000, training accuracy 0.402542 test accuracy 0.378784
step 2000, training accuracy 0.509338 test accuracy 0.551205
step 3000, training accuracy 0.845022 test accuracy 0.824984
step 4000, training accuracy 0.928436 test accuracy 0.914872
step 5000, training accuracy 0.9505 test accuracy 0.933533
step 6000, training accuracy 0.948405 test accuracy 0.948905
step 7000, training accuracy 0.963318 test accuracy 0.940776
step 8000, training accuracy 0.953124 test accuracy 0.943543
step 9000, training accuracy 0.920974 test accuracy 0.949701
test accuracy 0.950582


In [119]:
zipped = list(zip(SEEDS, [rev_comp(seed) for seed in SEEDS], [SEQ_INFO.loc[seed]['mirna_seq'] for seed in SEEDS]))
features, labels, sequences, stypes, lens = [], [], [], [], []
l8, l7, l71a = [], [], []

i = 0
for row in logFCs.iterrows():
    utr = UTRS.loc[row[0]]['UTR sequence']
    row = row[1]
    for seed, rev, mirna in zipped:
        val = row[seed]
        if np.isnan(val):
            continue
        elif utr.count(rev[1:]) == 1:
            loc = utr.find(rev[1:])
            if (loc-15) >= 0:
                if (loc + 9) < len(utr):
                    i += 1
                    seq1, seq2 = mirna[:20], utr[loc-15:loc+9]

                    # only take better than 6mer
                    if (rev + 'A') in seq2:
                        stypes.append(-0.164921079958)
                        l8.append(val)
                    elif (rev) in seq2:
                        stypes.append(-0.0726948723592)
                        l7.append(val)
                    elif (rev[1:] + 'A') in seq2:
                        stypes.append(-0.0474966301044)
                        l71a.append(val)
                    else:
                        continue
                    
                    sequences.append((seq1, seq2))
                    labels.append(val)
                    lens.append(row['UTR length'])

print(i, len(labels), len(sequences), len(lens))

with open('../data/logfc_7mer_8mer_extra.txt', 'w') as f:
    for seqs, label, utrlen in zip(sequences, labels, lens):
        f.write('{},{},{},{:.4}\n'.format(seqs[0], seqs[1], label, np.log10(utrlen)))

77423 35863 35863 35863


In [125]:
dim1, dim2 = 80, 96
num_output1 = 16
num_output2 = 16
fully_connected_nodes = 32
out_nodes = 1

NN = NeuralNet2D(sess, dim1, dim2, out_nodes)
NN.add_convolution(4, 4, 4, 4, num_output1)
NN.add_convolution(2, 2, 1, 1, num_output2)
NN.add_fully_connected(fully_connected_nodes)
NN.add_dropout(out_nodes)
NN.make_train_step('regression')
NN.train_model(logfc_train, logfc_test, num_epoch=10000,
               batch_size=100, report_int=1000, keep_prob_train=0.5)

step 0, training accuracy -3.45594 test accuracy -2.44972
step 1000, training accuracy 0.000285447 test accuracy 0.00501722
step 2000, training accuracy 0.0139347 test accuracy 0.0031606
step 3000, training accuracy 0.0388749 test accuracy 0.0246275
step 4000, training accuracy 0.0674052 test accuracy 0.0304471
step 5000, training accuracy 0.0413511 test accuracy 0.0370678
step 6000, training accuracy -0.0415183 test accuracy 0.0389867
step 7000, training accuracy 0.0528648 test accuracy 0.0308995
step 8000, training accuracy 0.116715 test accuracy 0.0487014
step 9000, training accuracy 0.0626391 test accuracy 0.0537782
test accuracy 0.0496244
